In [1]:
using Pkg
# Pkg.activate("/home/mat/.julia/environments/v1.5/Project.toml")
Pkg.activate("/media/mat/HDD/QROMP/")

 Activating environment at `/media/mat/HDD/QROMP/Project.toml`


In [9]:
using Revise
using LinearAlgebra
using Statistics
using BenchmarkTools
using QROMP

In [19]:
ψ = randn(100, 20)
ϕ = randn(100)
u = randn(100)
c = ψ\u;

In [20]:
# https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/qr.jl
function qrfactUnblocked!(A::AbstractMatrix{T}) where {T}
    LinearAlgebra.require_one_based_indexing(A)
    m, n = size(A)
    τ = zeros(T, min(m,n))
    for k = 1:min(m - 1 + !(T<:Real), n)
        x = view(A, k:m, k)
        τk = LinearAlgebra.reflector!(x)
        τ[k] = τk
        LinearAlgebra.reflectorApply!(x, τk, view(A, k:m, k + 1:n))
    end
        QR(A, τ)
end

function qrfactUnblocked(A::AbstractMatrix{T}, arg...; kwargs...) where T
    LinearAlgebra.require_one_based_indexing(A)
    AA = similar(A, LinearAlgebra._qreltype(T), size(A))
    copyto!(AA, A)
    return qrfactUnblocked!(AA, arg...; kwargs...)
end

qrfactUnblocked (generic function with 1 method)

In [24]:
F = qrfactUnblocked(ψ);

In [25]:
G = qrfactUnblocked(hcat(ψ, ϕ))

QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.105128    0.0116581    -0.0233742   …   0.209445     0.0835858
  0.0622341  -0.0387614    -0.112149        0.143246     0.0121464
 -0.0244353  -0.0197633    -0.163663        0.0190582   -0.0541857
  0.114228   -0.0607459    -0.0472817      -0.0344992   -0.103191
 -0.0570639   0.0702791    -0.014658       -0.0924599   -0.0703848
 -0.0151145  -0.0598913    -0.0651648   …   0.0794288    0.0444461
  0.097539    0.0381092     0.00700252     -0.125371     0.175088
  0.0347226   0.153019     -0.0636664       0.134639    -0.095581
 -0.194536    0.12148       0.0246794      -0.112246     0.130593
 -0.0760138   0.266138      0.213872        0.0730843   -0.146082
 -0.0212348   0.000499164  -0.0452395   …  -0.107464     0.0446546
  0.0153674   0.0991384     0.0357812       0.0271459   -0.106452
  0.104009    0.10853      -0.00567157     -0.0231728   -0.0928744
  ⋮                                  

In [26]:
@btime begin 
    F = qrfactUnblocked(ψ)
end

  22.760 μs (3 allocations: 16.02 KiB)


QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.105128    0.0116581    -0.0233742   …   0.256743      0.124149
  0.0622341  -0.0387614    -0.112149        0.134235      0.00441805
 -0.0244353  -0.0197633    -0.163663       -0.00213451   -0.0723607
  0.114228   -0.0607459    -0.0472817      -0.034657     -0.103326
 -0.0570639   0.0702791    -0.014658       -0.0615544    -0.04388
 -0.0151145  -0.0598913    -0.0651648   …   0.0645549     0.0316902
  0.097539    0.0381092     0.00700252     -0.101303      0.195729
  0.0347226   0.153019     -0.0636664       0.161561     -0.0724923
 -0.194536    0.12148       0.0246794      -0.143243      0.10401
 -0.0760138   0.266138      0.213872        0.0552352    -0.16139
 -0.0212348   0.000499164  -0.0452395   …  -0.066812      0.0795179
  0.0153674   0.0991384     0.0357812       0.0405218    -0.0949811
  0.104009    0.10853      -0.00567157      0.0110062    -0.0635623
  ⋮                       

In [27]:
@btime begin 
    F = qrfactUnblocked(ψ)
    Gnew = updateqrfactUnblocked!(F, ϕ)
end

  25.569 μs (12 allocations: 66.73 KiB)


QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.105128    0.0116581    -0.0233742   …   0.209445     0.0835858
  0.0622341  -0.0387614    -0.112149        0.143246     0.0121464
 -0.0244353  -0.0197633    -0.163663        0.0190582   -0.0541857
  0.114228   -0.0607459    -0.0472817      -0.0344992   -0.103191
 -0.0570639   0.0702791    -0.014658       -0.0924599   -0.0703848
 -0.0151145  -0.0598913    -0.0651648   …   0.0794288    0.0444461
  0.097539    0.0381092     0.00700252     -0.125371     0.175088
  0.0347226   0.153019     -0.0636664       0.134639    -0.095581
 -0.194536    0.12148       0.0246794      -0.112246     0.130593
 -0.0760138   0.266138      0.213872        0.0730843   -0.146082
 -0.0212348   0.000499164  -0.0452395   …  -0.107464     0.0446546
  0.0153674   0.0991384     0.0357812       0.0271459   -0.106452
  0.104009    0.10853      -0.00567157     -0.0231728   -0.0928744
  ⋮                                  

In [36]:
F = qrfactUnblocked(ψ)
updateqrfactUnblocked!(F, ϕ)

QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.105128    0.0116581    -0.0233742   …   0.209445     0.0835858
  0.0622341  -0.0387614    -0.112149        0.143246     0.0121464
 -0.0244353  -0.0197633    -0.163663        0.0190582   -0.0541857
  0.114228   -0.0607459    -0.0472817      -0.0344992   -0.103191
 -0.0570639   0.0702791    -0.014658       -0.0924599   -0.0703848
 -0.0151145  -0.0598913    -0.0651648   …   0.0794288    0.0444461
  0.097539    0.0381092     0.00700252     -0.125371     0.175088
  0.0347226   0.153019     -0.0636664       0.134639    -0.095581
 -0.194536    0.12148       0.0246794      -0.112246     0.130593
 -0.0760138   0.266138      0.213872        0.0730843   -0.146082
 -0.0212348   0.000499164  -0.0452395   …  -0.107464     0.0446546
  0.0153674   0.0991384     0.0357812       0.0271459   -0.106452
  0.104009    0.10853      -0.00567157     -0.0231728   -0.0928744
  ⋮                                  

In [39]:
F.factors

100×20 Array{Float64,2}:
  9.92614     0.0354303    …  -0.0639473  -1.44299      1.44418
 -0.0563139  10.2378           1.77436     0.0310531   -0.485287
  0.0221108   0.0192862       -1.1939      1.50633      1.0431
 -0.103362    0.0573553        0.835411   -0.151797    -1.16709
  0.0516355  -0.0671195        1.71358    -0.385226    -1.51669
  0.0136767   0.0578466    …   2.02197     0.431086    -0.41065
 -0.0882604  -0.0377015       -0.98701    -0.624098    -0.710156
 -0.0314195  -0.147755        -1.07098     0.69056      0.057372
  0.17603    -0.115044         0.682221   -0.399738     0.132499
  0.0687828  -0.255597         2.38095     0.644888    -0.968703
  0.0192148  -0.000265057  …   0.293381   -0.0788016    0.173381
 -0.0139056  -0.0956556        1.76053     1.21213      0.984922
 -0.0941151  -0.105603        -2.03011    -0.91895     -0.236084
  ⋮                        ⋱                           
  0.118552   -0.0970128       -0.0827953  -0.183408    -0.088193
 -0.0219449  -0

In [31]:
G.τ-Gnew.τ

21-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -2.220446049250313e-16

In [115]:
Gnew.factors[:,21]

100-element Array{Float64,1}:
 -0.488413245949013
 -0.2408342290467529
  0.38362659581276326
  0.30437814099264493
 -0.32618583586434013
  0.4567962794090623
  1.2225595003563046
 -0.7216093574149086
 -1.1442320903156027
 -0.46734869429372217
 -1.5808087500785624
 -0.08940810244060465
 -0.5307731480532676
  ⋮
  0.006297018389183372
 -0.12713089298797434
  0.009041774802728211
  0.053845723219263675
  0.04200187544678161
  0.12824677186187136
  0.03897154166277803
  0.04161913552518682
 -0.03181302095435804
  0.25899770135199457
  0.04252163024276664
  0.02681467556234496

In [114]:
G.factors-Gnew.factors

100×21 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  -5.55112e-17
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0   2.22045e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0   5.55112e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  -1.11022e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  -4.44089e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  -3.33067e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  -7.77156e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0   2.22045e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  -8.88178e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0   8.32667e-17
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0   4.

In [105]:
G.τ

21-element Array{Float64,1}:
 1.0025721565346326
 1.074674408002386
 1.1116106180492846
 1.0305006242425805
 1.1869360022511946
 1.0232061339577279
 1.003483118244591
 1.0153981674037482
 1.0564403077048516
 1.3239513071534774
 1.0437367339084225
 1.1014263788576575
 1.1515370156757803
 1.0537852089561568
 1.099118292848891
 1.0881571284805598
 1.0543588324102235
 1.1158342125077252
 1.004894373903045
 1.0575243240281342
 1.0355377689024232

In [115]:
F = qr(ψ);

In [116]:
ek = vcat(zeros(99), [1.0]);

In [117]:
# https://discourse.julialang.org/t/extract-submatrix-from-qr-factor-is-slow-julia-1-4/36582/4
# We want to avoid to form the Q matrix, instead, we want to leverage the Householder reflections
# For steps 6. and 11. of Baptista et al. 2019
@time q = (F.Q*ek) #2.852 μs (4 allocations: 1.17 KiB)
@time (q'*u)*q #102.923 ns (3 allocations: 928 bytes)
@time ((F.Q[:,end])'*u)*F.Q[:,end] #559.818 μs (610 allocations: 399.67 KiB);

  0.000030 seconds (4 allocations: 1.172 KiB)
  0.000008 seconds (3 allocations: 928 bytes)
  0.000656 seconds (611 allocations: 399.688 KiB)


In [118]:
norm(Matrix(F.Q)'*ψ-F.R)

1.895054461915781e-14

In [119]:
fieldnames(typeof(F))

(:factors, :T)

In [120]:
F.factors

100×20 Array{Float64,2}:
 -9.79277      -0.984645    …  -0.178133   -0.920504   -0.0135803
  0.0437092    10.685           2.09934     0.521827   -0.648501
  0.042276      0.105627       -1.75699    -0.376509   -0.21701
  0.0245851    -0.00224957      0.215839    0.199624   -0.760037
  0.226206      0.00291478     -0.23315     0.215037    0.476057
  0.0280383     0.0293287   …  -0.436564    0.384986   -0.0363558
 -0.11976      -0.142948       -0.612313   -0.751825   -0.958349
  0.0987301     0.0096388      -1.41889    -1.00214    -0.303064
  0.00664053    0.0782998      -1.34945    -1.17577    -0.777213
 -0.223901      0.0603178      -1.09634    -1.52535    -0.797304
  0.0612526    -0.103657    …   0.283859   -0.209914    1.1728
 -0.066061      0.0268233       0.950081   -1.55397     0.523886
  0.000444714  -0.0623628       0.218406   -0.279034   -0.882502
  ⋮                         ⋱                          
 -0.0649167    -0.0909868       0.0297019   0.0242575   0.151225
 -0.081974

In [136]:
?LinearAlgebra.reflector!

No documentation found.

`LinearAlgebra.reflector!` is a `Function`.

```
# 1 method for generic function "reflector!":
[1] reflector!(x::AbstractArray{T,1} where T) in LinearAlgebra at /home/mat/julia-1.5.3/share/julia/stdlib/v1.5/LinearAlgebra/src/generic.jl:1467
```


In [137]:
?LinearAlgebra.reflectorApply!

No documentation found.

`LinearAlgebra.reflectorApply!` is a `Function`.

```
# 1 method for generic function "reflectorApply!":
[1] reflectorApply!(x::AbstractArray{T,1} where T, τ::Number, A::StridedArray{T, 2} where T) in LinearAlgebra at /home/mat/julia-1.5.3/share/julia/stdlib/v1.5/LinearAlgebra/src/generic.jl:1491
```


In [302]:
F = qrfactUnblocked(ψ);

k = 1
k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
k = 16
k = 17
k = 18
k = 19
k = 20


In [303]:
G = qrfactUnblocked(hcat(ψ, ϕ))

k = 1
k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
k = 16
k = 17
k = 18
k = 19
k = 20
k = 21


QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.110278    -0.28482     -0.0808827   …   0.112565    0.0605037
 -0.0550092   -0.0387661    0.0411486       0.070769   -0.215491
 -0.114262    -0.195025    -0.203973        0.0438424   0.0464974
  0.166628     0.180783    -0.0278729      -0.0110039   0.0783531
  0.117406    -0.151044    -0.0576269      -0.0621247   0.153467
  0.0625153   -0.0234599   -0.0296683   …   0.0879424   0.0893787
 -0.0874471    0.0209421   -0.257712        0.0557685  -0.00558999
 -0.0943372    0.129629    -0.221895       -0.0136556   0.0644737
  0.041136    -0.106126     0.222356        0.0962503  -0.141157
  0.1004      -0.0540468   -0.198087       -0.0892233  -0.0122304
 -0.196271     0.218932     0.0126177   …  -0.150046    0.000790179
  0.0705165    0.0117309   -0.0538593      -0.0982075   0.100268
 -0.103496    -0.0276181   -0.112628        0.0791877   0.0235514
  ⋮                                     ⋱    

In [296]:
G.τ

21-element Array{Float64,1}:
 1.1102781392580394
 1.0246546185521208
 1.2029516843072843
 1.1888963930430616
 1.039696281491836
 1.0304165917739927
 1.0336314017857533
 1.1270509388996595
 1.0174301953706921
 1.0567364709644518
 1.2532032647131552
 1.0121871248585186
 1.0705874493549707
 1.02971705524492
 1.2925360222644808
 1.1392994868375785
 1.0685179307695711
 1.1610110567894871
 1.0302393390412694
 1.0495366629229028
 1.0881974894188031

In [286]:
updateqrfactUnblocked!(G, ϕϕ)

LoadError: [91mMethodError: no method matching setindex!(::Float64, ::Float64, ::Int64)[39m

In [234]:
updateqrfactUnblocked!

QR{Float64,Array{Float64,2}}

In [254]:
G.factors

100×20 Array{Float64,2}:
 -10.7834      -0.896036    …  -0.422551    0.374794    0.701275
   0.0495454   10.4772         -0.542871   -1.62349     0.683029
   0.102913     0.161726       -0.288658   -0.811318    0.938689
  -0.150078    -0.134717        0.853594   -1.49164    -0.753418
  -0.105745     0.176803       -0.264071    1.04816    -1.31395
  -0.056306     0.0385467   …  -0.480679   -2.41054    -1.20348
   0.0787614   -0.0423313       0.0515569  -0.242529   -1.93923
   0.0849672   -0.150128        0.0891922   0.0283761   1.22886
  -0.0370501    0.113871       -1.14125    -0.38301     0.106142
  -0.0904276    0.0778822      -2.05463     2.52724    -1.21748
   0.176776    -0.262802    …  -0.0812119   1.14778     1.25183
  -0.0635125    0.00620571      1.35907    -0.0900456   0.595865
   0.0932167    0.00104246      0.239759    0.283368   -0.480085
   ⋮                        ⋱                          
  -0.123631    -0.042236       -0.0608244  -0.0372634  -0.0379216
   0.059263   

In [258]:
@time hcat(G.factors, zeros(100)')

LoadError: [91mArgumentError: number of rows of each array must match (got (100, 1))[39m

In [252]:
?hcat

search: hcat hvcat MathConstants SphericalDirections @threadcall catch_backtrace



```
hcat(A...)
```

Concatenate along dimension 2.

# Examples

```jldoctest
julia> a = [1; 2; 3; 4; 5]
5-element Array{Int64,1}:
 1
 2
 3
 4
 5

julia> b = [6 7; 8 9; 10 11; 12 13; 14 15]
5×2 Array{Int64,2}:
  6   7
  8   9
 10  11
 12  13
 14  15

julia> hcat(a,b)
5×3 Array{Int64,2}:
 1   6   7
 2   8   9
 3  10  11
 4  12  13
 5  14  15

julia> c = ([1; 2; 3], [4; 5; 6])
([1, 2, 3], [4, 5, 6])

julia> hcat(c...)
3×2 Array{Int64,2}:
 1  4
 2  5
 3  6

julia> x = Matrix(undef, 3, 0)  # x = [] would have created an Array{Any, 1}, but need an Array{Any, 2}
3×0 Array{Any,2}

julia> hcat(x, [1; 2; 3])
3×1 Array{Any,2}:
 1
 2
 3
```


In [250]:
push!(G.factors, zeros(100))

LoadError: [91mMethodError: no method matching push!(::Array{Float64,2}, ::Array{Float64,1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  push!(::Any, ::Any, [91m::Any[39m) at abstractarray.jl:2252[39m
[91m[0m  push!(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at abstractarray.jl:2253[39m
[91m[0m  push!([91m::BenchmarkTools.Trial[39m, ::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at /home/mat/.julia/packages/BenchmarkTools/eCEpo/src/trials.jl:25[39m
[91m[0m  ...[39m

In [240]:
push!(G.τ, 1.0)

21-element Array{Float64,1}:
 1.1102781392580394
 1.0246546185521208
 1.2029516843072843
 1.1888963930430616
 1.039696281491836
 1.0304165917739927
 1.0336314017857533
 1.1270509388996595
 1.0174301953706921
 1.0567364709644518
 1.2532032647131552
 1.0121871248585186
 1.0705874493549707
 1.02971705524492
 1.2925360222644808
 1.1392994868375785
 1.0685179307695711
 1.1610110567894871
 1.0302393390412694
 1.0495366629229028
 1.0

In [236]:
qrfactUnblocked(zeros(0,0))

QR{Float64,Array{Float64,2}}
Q factor:
0×0 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}
R factor:
0×0 Array{Float64,2}

In [230]:
ψψ = copy(ψ)
G = qrfactUnblocked!(ψψ)

k = 1
k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
k = 16
k = 17
k = 18
k = 19
k = 20


QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.110278    -0.28482     -0.0808827   …   0.109822      0.0620089
 -0.0550092   -0.0387661    0.0411486       0.065581     -0.212644
 -0.114262    -0.195025    -0.203973        0.0343081     0.0517293
  0.166628     0.180783    -0.0278729      -0.000549353   0.0726162
  0.117406    -0.151044    -0.0576269      -0.101173      0.174895
  0.0625153   -0.0234599   -0.0296683   …   0.0748155     0.0965819
 -0.0874471    0.0209421   -0.257712        0.0501701    -0.00251792
 -0.0943372    0.129629    -0.221895       -0.0224587     0.0693043
  0.041136    -0.106126     0.222356        0.0875784    -0.136398
  0.1004      -0.0540468   -0.198087       -0.0874815    -0.0131862
 -0.196271     0.218932     0.0126177   …  -0.126125     -0.0123359
  0.0705165    0.0117309   -0.0538593      -0.0718575     0.0858084
 -0.103496    -0.0276181   -0.112628        0.0634922     0.0321642
  ⋮                 

In [229]:
ψψ

100×20 Array{Float64,2}:
 -11.9726      1.98925      0.0805942  …   0.3565    -0.821361   2.4014
  -0.543641   10.834        1.21408       -1.79861   -1.38959    1.06527
  -1.12922     2.10265    -12.8549        -1.24325   -0.122432   1.05776
   1.64674    -1.8795      -0.515225       0.410145  -3.16703   -0.744448
   1.16029     1.86452     -0.538019       0.46609   -0.254964  -1.34766
   0.617822    0.340356    -0.318401   …  -0.732821  -1.93115   -0.148407
  -0.864216   -0.3401      -2.54526       -1.76795   -0.518522  -2.23154
  -0.932309   -1.5928      -2.26233        1.13753    0.886431   0.981135
   0.406535    1.26262      2.2822        -1.49682   -0.072447   2.16767
   0.992224    0.7341      -2.00732       -4.10867    3.51399   -1.63177
  -1.93969    -2.73245      0.0850079  …  -0.270928   3.11432    1.43077
   0.696896   -0.0535155   -0.591821       1.55899    1.663      0.893739
  -1.02283     0.197665    -1.04514       -0.968061  -0.14865   -0.608136
   ⋮                  

In [227]:
Gnew = qrfactUnblocked(hcat(ψ, ϕ))

k = 1
k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
k = 16
k = 17
k = 18
k = 19
k = 20
k = 21


QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.110278    -0.28482     -0.0808827   …   0.112565    0.0605037
 -0.0550092   -0.0387661    0.0411486       0.070769   -0.215491
 -0.114262    -0.195025    -0.203973        0.0438424   0.0464974
  0.166628     0.180783    -0.0278729      -0.0110039   0.0783531
  0.117406    -0.151044    -0.0576269      -0.0621247   0.153467
  0.0625153   -0.0234599   -0.0296683   …   0.0879424   0.0893787
 -0.0874471    0.0209421   -0.257712        0.0557685  -0.00558999
 -0.0943372    0.129629    -0.221895       -0.0136556   0.0644737
  0.041136    -0.106126     0.222356        0.0962503  -0.141157
  0.1004      -0.0540468   -0.198087       -0.0892233  -0.0122304
 -0.196271     0.218932     0.0126177   …  -0.150046    0.000790179
  0.0705165    0.0117309   -0.0538593      -0.0982075   0.100268
 -0.103496    -0.0276181   -0.112628        0.0791877   0.0235514
  ⋮                                     ⋱    

In [218]:
@time G = qrfactUnblocked(ψ)

  0.000027 seconds (3 allocations: 16.016 KiB)


QR{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.110278    -0.28482     -0.0808827   …   0.109822      0.0620089
 -0.0550092   -0.0387661    0.0411486       0.065581     -0.212644
 -0.114262    -0.195025    -0.203973        0.0343081     0.0517293
  0.166628     0.180783    -0.0278729      -0.000549353   0.0726162
  0.117406    -0.151044    -0.0576269      -0.101173      0.174895
  0.0625153   -0.0234599   -0.0296683   …   0.0748155     0.0965819
 -0.0874471    0.0209421   -0.257712        0.0501701    -0.00251792
 -0.0943372    0.129629    -0.221895       -0.0224587     0.0693043
  0.041136    -0.106126     0.222356        0.0875784    -0.136398
  0.1004      -0.0540468   -0.198087       -0.0874815    -0.0131862
 -0.196271     0.218932     0.0126177   …  -0.126125     -0.0123359
  0.0705165    0.0117309   -0.0538593      -0.0718575     0.0858084
 -0.103496    -0.0276181   -0.112628        0.0634922     0.0321642
  ⋮                 

In [219]:
fieldnames(typeof(G))

(:factors, :τ)

In [223]:
G.factors

100×20 Array{Float64,2}:
 -10.7834      -0.896036    …  -0.422551    0.374794    0.701275
   0.0495454   10.4772         -0.542871   -1.62349     0.683029
   0.102913     0.161726       -0.288658   -0.811318    0.938689
  -0.150078    -0.134717        0.853594   -1.49164    -0.753418
  -0.105745     0.176803       -0.264071    1.04816    -1.31395
  -0.056306     0.0385467   …  -0.480679   -2.41054    -1.20348
   0.0787614   -0.0423313       0.0515569  -0.242529   -1.93923
   0.0849672   -0.150128        0.0891922   0.0283761   1.22886
  -0.0370501    0.113871       -1.14125    -0.38301     0.106142
  -0.0904276    0.0778822      -2.05463     2.52724    -1.21748
   0.176776    -0.262802    …  -0.0812119   1.14778     1.25183
  -0.0635125    0.00620571      1.35907    -0.0900456   0.595865
   0.0932167    0.00104246      0.239759    0.283368   -0.480085
   ⋮                        ⋱                          
  -0.123631    -0.042236       -0.0608244  -0.0372634  -0.0379216
   0.059263   

In [222]:
I + tril(G.factors, -1)

LoadError: [91mDimensionMismatch("matrix is not square: dimensions are (100, 20)")[39m

In [220]:
G.factors

100×20 Array{Float64,2}:
 -10.7834      -0.896036    …  -0.422551    0.374794    0.701275
   0.0495454   10.4772         -0.542871   -1.62349     0.683029
   0.102913     0.161726       -0.288658   -0.811318    0.938689
  -0.150078    -0.134717        0.853594   -1.49164    -0.753418
  -0.105745     0.176803       -0.264071    1.04816    -1.31395
  -0.056306     0.0385467   …  -0.480679   -2.41054    -1.20348
   0.0787614   -0.0423313       0.0515569  -0.242529   -1.93923
   0.0849672   -0.150128        0.0891922   0.0283761   1.22886
  -0.0370501    0.113871       -1.14125    -0.38301     0.106142
  -0.0904276    0.0778822      -2.05463     2.52724    -1.21748
   0.176776    -0.262802    …  -0.0812119   1.14778     1.25183
  -0.0635125    0.00620571      1.35907    -0.0900456   0.595865
   0.0932167    0.00104246      0.239759    0.283368   -0.480085
   ⋮                        ⋱                          
  -0.123631    -0.042236       -0.0608244  -0.0372634  -0.0379216
   0.059263   

In [221]:
G.τ

20-element Array{Float64,1}:
 1.1102781392580394
 1.0246546185521208
 1.2029516843072843
 1.1888963930430616
 1.039696281491836
 1.0304165917739927
 1.0336314017857533
 1.1270509388996595
 1.0174301953706921
 1.0567364709644518
 1.2532032647131552
 1.0121871248585186
 1.0705874493549707
 1.02971705524492
 1.2925360222644808
 1.1392994868375785
 1.0685179307695711
 1.1610110567894871
 1.0302393390412694
 1.0495366629229028

In [159]:
x = randn(5)

5-element Array{Float64,1}:
  2.230257488227875
 -0.11888208132838382
  0.9750242283602661
  0.9194231693943637
  0.8783439956829767

In [ ]:
# https://people.inf.ethz.ch/gander/papers/qrneu.pdf
function qr_householder(A::AbstractMatrix{T}) where {T}
    m, n = size(A)
    R = deepcopy(A)
    factors = zeros()
    for j = 1:n
        s = 0
        for i = j:m
            s += A[i,j]^2
            s = sqrt(s)
            if A[j,j]>0
                d = -s
            else
                d = s
            end
            fact = sqrt(s*(s+abs(A[jj])))
            A[j,j] -= d
            for k = j:m
                A[k,j] *=(1/fact)
            end
            for
        x = view(A, k:m, k)
        v = sign(x[1])*norm(x)
        τ[k] = τk
        LinearAlgebra.reflectorApply!(x, τk, view(A, k:m, k + 1:n))
    end
    
    

In [176]:
ψ

100×20 Array{Float64,2}:
  9.78429      -0.48908        0.902408     …  -0.542897     -1.3036
  0.00510913   -9.75913       -1.78364         -1.11403       0.116365
 -0.00510463    0.00222295   -11.1361           1.29031      -0.17273
  0.00272449   -0.00580306     0.00595295       0.379762      0.536234
  0.00134546   -0.00421924    -0.00206149      -1.8216        0.896753
  0.00977582    0.000901351    0.00163762   …   0.662662      1.44236
 -0.00156515    0.00213024    -0.00222244       0.373323      1.84818
  0.000889446  -0.00539386     0.00590559      -1.59019      -0.442572
 -0.00902239    0.00186711    -0.00420206       0.169216      1.45984
  0.00511326   -0.00076226    -0.00647957      -0.813985      1.32556
 -0.00321999    0.000621044   -0.000100839  …  -0.591463     -1.25288
 -0.00167035   -0.00662055     0.000687231     -0.595211      0.0305723
 -0.00500827    0.00300516    -0.00745038      -0.984268     -0.0656236
  ⋮                                         ⋱             

In [168]:
G.factors

10×10 Array{Float64,2}:
 -2.84393     0.246886    0.0446929  …  -0.37811    1.181       -0.404545
 -0.203868   -3.8533     -0.894796      -0.8649    -0.150774     0.784774
  0.166684   -0.208803   -1.91641       -0.686178   0.895284     0.783668
 -0.46087    -0.151256   -0.270116       0.767728   1.5065       0.524178
  0.0667787   0.150098    0.391511       0.791301   0.0276143   -1.12165
  0.303348    0.505618    0.314508   …   0.60899   -1.07365      0.934828
  0.302819   -0.312443    0.456795       0.856288  -0.00879321  -0.117157
 -0.36252    -0.349202   -0.138452      -1.20988   -0.909806    -0.854999
  0.0302492   0.399715   -0.138697      -0.279071   2.22412     -0.842169
  0.301023   -0.0299667  -0.467326      -0.190066  -0.976469     0.620365

In [169]:
F

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0705955    -0.100639    -0.210567    …  -0.0732656    0.0603258
 -0.0467948    -0.100197    -0.0408091      -0.0210045    0.154132
 -0.0452604    -0.12        -0.0164645      -0.0523336    0.126038
 -0.0263207    -9.14915e-6  -0.0811754      -0.0402525   -0.00762209
 -0.242176     -0.0259593    0.0388997      -0.0272436    0.0369929
 -0.0300177    -0.0349601    0.113258    …   0.111117    -0.100995
  0.128214      0.168695     0.129086        0.00911191  -0.128959
 -0.1057       -0.0204983    0.123006       -0.0199709    0.0500355
 -0.00710932   -0.086469    -0.0275212      -0.0455833   -0.0162662
  0.239707     -0.0435629    0.0595007      -0.014942    -0.0199145
 -0.0655767     0.107422    -0.0173348   …   0.0027334   -0.244812
  0.0707247    -0.0227446    0.10327         0.0749076   -0.159555
 -0.000476109   0.0682923    0.170775       -0.00983738  -0.04858

In [160]:
τ = LinearAlgebra.reflector!(x)

1.811368616163854

In [151]:
?LAPACK.geqrt!

```
geqrt!(A, T)
```

Compute the blocked `QR` factorization of `A`, `A = QR`. `T` contains upper triangular block reflectors which parameterize the elementary reflectors of the factorization. The first dimension of `T` sets the block size and it must be between 1 and `n`. The second dimension of `T` must equal the smallest dimension of `A`.

Returns `A` and `T` modified in-place.

---

```
geqrt!(A, nb) -> (A, T)
```

Compute the blocked `QR` factorization of `A`, `A = QR`. `nb` sets the block size and it must be between 1 and `n`, the second dimension of `A`.

Returns `A`, modified in-place, and `T`, which contains upper triangular block reflectors which parameterize the elementary reflectors of the factorization.


In [123]:
Fnew = qr(hcat(ψ, ϕ))

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0705955    -0.100639    -0.210567    …  -0.0703152     0.0322968
 -0.0467948    -0.100197    -0.0408091      -0.0196483     0.141247
 -0.0452604    -0.12        -0.0164645      -0.0480847     0.0856732
 -0.0263207    -9.14915e-6  -0.0811754      -0.0450948     0.0383802
 -0.242176     -0.0259593    0.0388997      -0.0248704     0.014448
 -0.0300177    -0.0349601    0.113258    …   0.108792     -0.078906
  0.128214      0.168695     0.129086        0.00255155   -0.0666351
 -0.1057       -0.0204983    0.123006       -0.0184123     0.0352282
 -0.00710932   -0.086469    -0.0275212      -0.046186     -0.0105406
  0.239707     -0.0435629    0.0595007      -0.013959     -0.0292539
 -0.0655767     0.107422    -0.0173348   …  -0.000269531  -0.216284
  0.0707247    -0.0227446    0.10327         0.0778024    -0.187055
 -0.000476109   0.0682923    0.170775       -0.006844

In [125]:
?householder

search:

Couldn't find householder
Perhaps you meant homedir


No documentation found.

Binding `householder` does not exist.


In [127]:
vh = deepcopy(ϕ[21:100])
vh[1] += sign(ϕ[21])*norm(ϕ[21:100])

-9.583593452178766

In [131]:
fieldnames(typeof(F))

(:factors, :T)

In [133]:
F.factors

100×20 Array{Float64,2}:
 -9.79277      -0.984645    …  -0.178133   -0.920504   -0.0135803
  0.0437092    10.685           2.09934     0.521827   -0.648501
  0.042276      0.105627       -1.75699    -0.376509   -0.21701
  0.0245851    -0.00224957      0.215839    0.199624   -0.760037
  0.226206      0.00291478     -0.23315     0.215037    0.476057
  0.0280383     0.0293287   …  -0.436564    0.384986   -0.0363558
 -0.11976      -0.142948       -0.612313   -0.751825   -0.958349
  0.0987301     0.0096388      -1.41889    -1.00214    -0.303064
  0.00664053    0.0782998      -1.34945    -1.17577    -0.777213
 -0.223901      0.0603178      -1.09634    -1.52535    -0.797304
  0.0612526    -0.103657    …   0.283859   -0.209914    1.1728
 -0.066061      0.0268233       0.950081   -1.55397     0.523886
  0.000444714  -0.0623628       0.218406   -0.279034   -0.882502
  ⋮                         ⋱                          
 -0.0649167    -0.0909868       0.0297019   0.0242575   0.151225
 -0.081974

In [122]:
F.Q

100×100 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0705955    -0.100639    -0.210567    …  -0.0732656    0.0603258
 -0.0467948    -0.100197    -0.0408091      -0.0210045    0.154132
 -0.0452604    -0.12        -0.0164645      -0.0523336    0.126038
 -0.0263207    -9.14915e-6  -0.0811754      -0.0402525   -0.00762209
 -0.242176     -0.0259593    0.0388997      -0.0272436    0.0369929
 -0.0300177    -0.0349601    0.113258    …   0.111117    -0.100995
  0.128214      0.168695     0.129086        0.00911191  -0.128959
 -0.1057       -0.0204983    0.123006       -0.0199709    0.0500355
 -0.00710932   -0.086469    -0.0275212      -0.0455833   -0.0162662
  0.239707     -0.0435629    0.0595007      -0.014942    -0.0199145
 -0.0655767     0.107422    -0.0173348   …   0.0027334   -0.244812
  0.0707247    -0.0227446    0.10327         0.0749076   -0.159555
 -0.000476109   0.0682923    0.170775       -0.00983738  -0.0485843
  ⋮                                      ⋱               
 

In [101]:
Fnew.R

21×21 Array{Float64,2}:
 10.1654   -1.18599  -0.232919  …  -0.607099   0.738939   -0.999455
  0.0     -10.3087    1.16229      -0.540267   1.20549     0.973458
  0.0       0.0      10.5791       -1.13928   -0.586131    0.95655
  0.0       0.0       0.0          -0.220456  -0.892879    1.42508
  0.0       0.0       0.0          -0.837836   0.751373   -0.773969
  0.0       0.0       0.0       …  -1.76798   -0.70448     0.779583
  0.0       0.0       0.0           1.66119   -1.00365     1.83056
  0.0       0.0       0.0           1.17933   -1.29004     1.04533
  0.0       0.0       0.0          -0.156345  -0.452466    1.46067
  0.0       0.0       0.0          -2.06553   -0.832744   -0.947424
  0.0       0.0       0.0       …  -0.183742   0.0195222   0.095289
  0.0       0.0       0.0          -1.68542    0.444276    2.54893
  0.0       0.0       0.0          -1.77112    2.56745    -1.3364
  0.0       0.0       0.0           1.27211    0.781254    0.700294
  0.0       0.0       0.0       

In [99]:
Matrix(Fnew.Q)

100×21 Array{Float64,2}:
 -0.179792     -0.0102689   -0.00872862  …  -0.0956888    0.00562993
 -0.01942      -0.091573    -0.153647        0.0964798   -0.135396
  0.0894703     0.244164    -0.195865       -0.0525083    0.0325773
 -0.157922      0.0187791    0.0691462      -0.219997    -0.124997
  0.000958288   0.0910368    0.213608       -0.0876106   -0.0455153
 -0.218258      0.0829915    0.0545718   …  -0.00969883   0.0910254
  0.0456609     0.0370957   -0.0086529       0.0324374    0.0296257
 -0.176313     -0.0255634   -0.0438317       0.190384     0.0111229
  0.0548072    -0.00144394   0.0753013      -0.0250942    0.0679212
 -0.0692097    -0.11416      0.233036       -0.00465007   0.115794
 -0.0056473    -0.0823523    0.0226756   …   0.143532    -0.0253104
 -0.272315     -0.071227     0.241264        0.196879     0.0225334
 -0.0584766    -0.0149938   -0.147795        0.00670103  -0.111517
  ⋮                                      ⋱                ⋮
 -0.148907     -0.0396372   -0.153

In [86]:
ϕ

100×1 Array{Float64,2}:
  1.488471947918379
  0.23151309105000684
  0.08342578565455548
 -0.15831262255049494
 -0.3611754442513597
  0.969637196370213
 -0.7399418504557957
  0.5499193637791298
 -1.2315522628319115
 -0.2911528778102039
 -1.8083582921316528
 -0.5321245926139485
  1.0490805510720536
  ⋮
  0.38770788588660143
 -0.5016589309589501
 -0.23040741085893024
  1.0787173760606186
  1.0328480441527197
  0.22145127269069317
 -0.46752260010386193
  0.8240911526103453
  0.08709844104061633
 -0.37929395192899745
 -1.2666828664149894
 -0.042241498251965284